In [1]:
from IPython.display import Markdown, display

date = '2022-04-05'
practice = '../_practice/' + date + '.md'
prepare =  '../_prepare/' + date + '.md'

# Why is the object file not human readable?  



## Prelude: REPL

One way we can use an interpreted language like python is to treat the interpreter like an application and then interact with it in the terminal.  



```Python
4+5
```

```
9

```

```Python
name = 'sarah'

```

```Python
name
```

```
'sarah'

```

```Python
[ord(char) for char in name]
```

```
[115, 97, 114, 97, 104]

```

```
[ord(char) for char in 'denno']
[100, 101, 110, 110, 111]

```

```
[ord(char) for char in 'Denno']
[68, 101, 110, 110, 111]

```

```
bytearray(34)
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

```

```
bytearray(3)
bytearray(b'\x00\x00\x00')

```

```
bytearray(5)
bytearray(b'\x00\x00\x00\x00\x00')

```

```
bytes(bytearray(3))
b'\x00\x00\x00'

```

```
with open("name.txt",'wb') as f:
...     f.write(bytes(bytearray(name))
...
... )
...
Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
TypeError: string argument without an encoding

```

``
`with open("name.txt",'wb') as f:
...     f.write(bytes(bytearray(name)))
...
Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
TypeError: string argument without an encoding

```

```
name_ints = [ord(char) for char in name]

```

```
with open("name.txt",'wb') as f:
...     f.write(bytes(bytearray(name_ints)))
...
5

```

```
exit()
(base) brownsarahm@~ $ cat name.txt
sarah(base) brownsarahm@~ $ python
Python 3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39)
[Clang 6.0 (clang-600.0.57)] on darwin
Type "help", "copyright", "credits" or "license" for more information.

```

```
with open("name.txt",'rb') as f:
...    print(f.read())
...
b'sarah'

```

```
with open("name.txt",'r') as f:
...    print(f.read())
...
sarah

```

```
with open("name.txt",'rb') as f:
...    content = f.read()
...

```

```
type(content)
<class 'bytes'>

```

```
f = open("name.text",'rb)
  File "<stdin>", line 1
    f = open("name.text",'rb)
                            ^
SyntaxError: EOL while scanning string literal

```

```
f = open("name.text",'rb')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
FileNotFoundError: [Errno 2] No such file or directory: 'name.text'

```

```
f = open("name.txt",'rb')

```

```
f.read()
b'sarah'

```

```
f.close()

```

```
bytes(bytearray(msg))
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'msg' is not defined

```

```
msg = [68, 114, 46, 32, 66, 114, 111, 119, 110,10]

```

```
bytes(bytearray(msg))
b'Dr. Brown\n'

```

```
3 >> 1
1

```

```
3 << 1
6

```

```
1 & 0
0

```

```
3 & 1
1

```

```
type(3)
<class 'int'>

```

```
3.4 & 4.5
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: unsupported operand type(s) for &: 'float' and 'float'

```

```
3 ^ 3
0

```

```
3 | 1
3

```

```
~3
-4

```

```
2**6
64

```

```
(54).bit_lenth()
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
AttributeError: 'int' object has no attribute 'bit_lenth'

```

```
(54).bit_length()
6

```

```
(5).bit_length()
3

```

```
5 & 1
1

```

```
bin(3)
'0b11'

```

```
bin(5)
'0b101'

```

```
bin(-5)
'-0b101'

```

```
b'1111'
b'1111'

```

```
0b11
3

```

```
0b111 & 0b0111
7

```

```
0b111 & 0b0100
4

```

```
bin(0b111 & 0b0100)
'0b100'

```

```
bin(0b1111 & 0b0100)
'0b100'

```

```
bin(0b1111 ^ 0b0100)
'0b1011'

```

```
bin(0b1111 - 0b0100)
'0b1011'

```

```
exit
Use exit() or Ctrl-D (i.e. EOF) to exit

```

```
exit()
(base) brownsarahm@~ $
```

(class19prep)=
## Prepare for next class
````{margin}
```{tip}
The text in () in these sections is an explanation of *why* that task is assigned
```
````


```{code-cell} ipython3
:tags: ["remove-input"]

display(Markdown(prepare))
```



(class19practice)=
## More Practice



```{code-cell} ipython3
:tags: ["remove-input"]
display(Markdown(practice))
```


## Questions After Class


Class Response Summary:



### When is the cutoff for getting work approved for the class?
```{toggle}
Our assigned final exam time is: DATE

So, until then you can get feedback and revise.  At that time, my assessment of your work will be final, and I'll evaluate your contract.
```

### Will the fact that we didn't have class the other week effect the amount of work we have to complete for our contracts?
```{toggle}
I will announce by how many you can reduce the required number of more practice/deeper exploration you can do.
```


• what are some programing implementations of this arithmetic

• Is there a low-level interface to actually make using bits as flags useful in python?

• whats the likely hood of me using bit operations in high level languages

• When would we use those binary operation (like in a job / research)?

### When using the bitwise operators (the &, |, etc.) what are the inputs?
```{toggle}
In Python, bit level operators are defined only for integers
```

• how often (if ever) do you find yourself working in a python shell like we did today? I find that to be extremely annoying and would rather take the time to make a .py file and use an IDE/run it from the terminal


### Questions to be answered later

- I have seen XOR gate used to solve certain programming problems efficiently, but I am curious about how to actually implement it